<a href="https://colab.research.google.com/github/NeyoxDrago/Algorithms-from-Scratch/blob/main/Decision%20Trees/Decision_Tree_Classifier_from_scratch_with_3_nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --header="Host: archive.ics.uci.edu" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.101 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://archive.ics.uci.edu/ml/machine-learning-databases/00422/" "https://archive.ics.uci.edu/ml/machine-learning-databases/00422/wifi_localization.txt" -c -O 'wifi_localization.txt'

--2021-01-06 06:47:56--  https://archive.ics.uci.edu/ml/machine-learning-databases/00422/wifi_localization.txt
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62000 (61K) [application/x-httpd-php]
Saving to: ‘wifi_localization.txt’

wifi_localization.t 100%[===================>]  60.55K  --.-KB/s    in 0.1s    

2021-01-06 06:47:56 (560 KB/s) - ‘wifi_localization.txt’ saved [62000/62000]



In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
import os

In [13]:
data = pd.read_csv("wifi_localization.txt",sep="\t",header=None)
data.columns = [f'feature_{i}' for i in data.columns]
target_column = data.columns.values[-1]
data

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7
0,-64,-56,-61,-66,-71,-82,-81,1
1,-68,-57,-61,-65,-71,-85,-85,1
2,-63,-60,-60,-67,-76,-85,-84,1
3,-61,-60,-68,-62,-77,-90,-80,1
4,-63,-65,-60,-63,-77,-81,-87,1
...,...,...,...,...,...,...,...,...
1995,-59,-59,-48,-66,-50,-86,-94,4
1996,-59,-56,-50,-62,-47,-87,-90,4
1997,-62,-59,-46,-65,-45,-87,-88,4
1998,-62,-58,-52,-61,-41,-90,-85,4


In [14]:
# sns.countplot(data['target'])

In [15]:
import math

def gini_impurity(arr):
  _,counts = np.unique(arr,return_counts=True)
  return 1 - np.sum((counts/len(arr))**2)

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test = train_test_split(data,test_size=0.2,random_state=99)

In [56]:
import scipy

class CustomDecisionTreeClassifier:

  def __init__(self,
               X,
               target_column="target",
               depth=0,
               max_depth=5):

    self.left = None
    self.center = None
    self.right = None
    self.data= X
    self.child_nodes = 3
    self.fkey = None
    self.max_depth = max_depth
    self.target_column = target_column
    self.depth = depth
    self.endpoints = None

    self.train(self.data)

    
  def __entropy(self,arr):
    values,counts = np.unique(arr,return_counts=True)
    probs = counts/len(arr)

    return -1*np.sum(probs*np.log2(probs))

  def __divide_data(self,df,column,
                    # fval
                    endpoints):
    
    if endpoints is None:    
      endpoints = np.linspace(df[column].min(),df[column].max(),self.child_nodes +1)

    x_left  = df[(df[column] <= endpoints[1])]
    x_center = df[(df[column] > endpoints[1]) & (df[column] <= endpoints[2])]
    x_right = df[(df[column] > endpoints[2])]

    return x_left,x_center,x_right


  def __information_gain(self,data,fkey,endpoints=None):
    
    left,center,right = self.__divide_data(data,fkey,endpoints)

    prob_l = float(left.shape[0]/data.shape[0])
    prob_c = float(center.shape[0]/data.shape[0])
    prob_r = float(right.shape[0]/data.shape[0])

    i_gain = self.__entropy(data[self.target_column].values) - (
         
        prob_l*self.__entropy(left[self.target_column].values) + 
        prob_c*(self.__entropy(center[self.target_column].values)) +
        prob_r*self.__entropy(right[self.target_column].values)
        
        )
        
    return i_gain
  
  def train(self,data):
    info_gains = []

    features = data.columns[:-1]

    for ix in features:
      info_gains.append(self.__information_gain(data,ix))
    
    index = np.argmax(info_gains)
    self.fkey = features[index]
    
    print(self.fkey,info_gains,index)
    
    # if sum(info_gains) == 0:
    #   self.target_val = int(np.ceil(data[self.target_column].mean()))
    #   return self

    self.endpoints = np.linspace(data[self.fkey].min(),data[self.fkey].max(),self.child_nodes + 1)
    
    left,center,right = self.__divide_data(data,self.fkey,self.endpoints)
    
    if left.shape[0] == 0 or right.shape[0] == 0 or center.shape[0] == 0:
      self.target_val = int(np.ceil(data[self.target_column].mean()))
      return 
    
    if self.depth >= self.max_depth :
      self.target_val = int(np.ceil(data[self.target_column].mean()))
      print("____")
      return 
    
    self.left = CustomDecisionTreeClassifier(left,depth=self.depth+1 ,max_depth = self.max_depth,target_column = self.target_column)
    self.center = CustomDecisionTreeClassifier(center,depth=self.depth+1 ,max_depth = self.max_depth,target_column = self.target_column)
    self.right = CustomDecisionTreeClassifier(right,depth=self.depth+1 ,max_depth = self.max_depth,target_column = self.target_column)

    self.target_val = int(np.ceil(data[self.target_column].mean()))
    return 

  def predict(self,test):

    if test[self.fkey] <= self.endpoints[1]:
      if self.left is None:
        return self.target_val
      
      return self.left.predict(test)
    
    elif (test[self.fkey] > self.endpoints[1]) & (test[self.fkey] <= self.endpoints[2]):
      if self.center is None:
        return self.target_val
      
      return self.center.predict(test)
    else:
      if self.right is None:
        return self.target_val
      
      return self.right.predict(test)

In [105]:
print(x_train.columns)
tree = CustomDecisionTreeClassifier(x_train,depth=1,max_depth=5,target_column=target_column)
tree

Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7'],
      dtype='object')
feature_0 [0.8870183649745533, 0.023671292103409902, 0.3156264850845967, 0.8073477395698803, 0.7070304528509734, 0.49723153178953683, 0.4276567102756128] 0
feature_4 [0.1680121293711505, 0.023823878267977383, 0.43956063348385954, 0.21763804389964814, 0.7421898017452545, 0.1363067642175093, 0.08510439160711902] 4
feature_3 [0.05788381337832543, 0.00527621170695311, 0.04730541326098728, 0.11248202402641701, 0.00886427019004532, 0.005596804911801506, 0.0009004771488978214] 3
feature_0 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0
feature_0 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0
____
feature_0 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0
____
feature_0 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0
____
feature_3 [0.01914462482941548, 0.0033251710469287543, 0.041295220473634986, 0.10701675809204175, 0.01679385377740658, 0.013080252808417653, 0.016598419150504584] 3
feature_0 

In [106]:
preds = []
for ix in range(x_test.shape[0]):
  preds.append(tree.predict(x_test.iloc[ix]))

from sklearn.metrics import accuracy_score
accuracy_score(x_test[target_column],preds)

0.8775

In [110]:
# def print_tree(g):
#   try:
#     if not g.left is None:
#       print_tree(g.left)
#       print_tree(g.center)
#       print_tree(g.right)

#     print(g.left.fkey,g.center.fkey,g.right.fkey)
#   except:
#      return 

# print_tree(tree)
# print(tree.fkey)

# g=tree
# print(g.fkey)
# print(g.left.fkey,g.center.fkey,g.right.fkey)
# for m,tree_name  in list(zip([g,g,g],['left','center','right'])):
#   print("\n",tree_name) 
#   for depth in range(2,6):
#     print(m.left.fkey,m.center.fkey,m.right.fkey)
    # m = getattr(m,tree_name)
  # break